In [4]:
from numpy import save, load
from pandas import read_csv
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
!pip install keras-tcn
from tcn import TCN

In [53]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testX.npy')
y_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [54]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [57]:
input_shape = x_train.shape[1:]
inp = Input(shape=x_train.shape[1:])
x = SpatialDropout1D(0.2)(inp)
x = TCN(128,dilations = [1, 2, 4], return_sequences=True)(x)
x = TCN(64,dilations = [1, 2, 4], return_sequences=True)(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
    
conc = concatenate([avg_pool, max_pool])
conc = Dense(64, activation="relu")(conc)
conc = Dropout(0.2)(conc)
outp = Dense(118, activation="softmax")(conc)    
model = Model(inputs=inp, outputs=outp)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 128, 6)]     0           []                               
                                                                                                  
 spatial_dropout1d_12 (SpatialD  (None, 128, 6)      0           ['input_13[0][0]']               
 ropout1D)                                                                                        
                                                                                                  
 tcn_24 (TCN)                   (None, 128, 128)     249728      ['spatial_dropout1d_12[0][0]']   
                                                                                                  
 tcn_25 (TCN)                   (None, 128, 64)      94656       ['tcn_24[0][0]']          

In [58]:
history = model.fit(x_train, y_train, epochs=50, verbose=True, validation_data=(x_validation, y_validation), batch_size=128)

Epoch 1/50
207/207 [==============================] - 26s 76ms/step - loss: 4.4323 - accuracy: 0.0857 - val_loss: 2.7955 - val_accuracy: 0.3691
Epoch 2/50
207/207 [==============================] - 15s 74ms/step - loss: 2.5319 - accuracy: 0.3537 - val_loss: 1.0794 - val_accuracy: 0.7692
Epoch 3/50
207/207 [==============================] - 15s 74ms/step - loss: 1.6739 - accuracy: 0.5436 - val_loss: 0.6122 - val_accuracy: 0.8573
Epoch 4/50
207/207 [==============================] - 15s 73ms/step - loss: 1.2930 - accuracy: 0.6471 - val_loss: 0.5058 - val_accuracy: 0.8911
Epoch 5/50
207/207 [==============================] - 15s 73ms/step - loss: 1.0609 - accuracy: 0.7093 - val_loss: 0.3762 - val_accuracy: 0.9097
Epoch 6/50
207/207 [==============================] - 15s 73ms/step - loss: 0.9360 - accuracy: 0.7437 - val_loss: 0.3449 - val_accuracy: 0.9142
Epoch 7/50
207/207 [==============================] - 15s 74ms/step - loss: 0.8362 - accuracy: 0.7680 - val_loss: 0.2686 - val_accuracy:

In [59]:
loss, accuracy = model.evaluate(x_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9742
Testing Accuracy: 0.9187
